In [ ]:
import os
from sentence_transformers import models, SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

In [ ]:
# 1. 모델 불러오기 (kykim BERT → SBERT 구조)
word_embedding_model = models.Transformer("./kykim_bert_kor_base_model")
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
def read_file(path):
    with open(path, "r", encoding="utf-8") as f:
        return [line.strip() for line in f if line.strip()]

In [ ]:
godae_ori = read_file(os.path.join("./고대/", "고대-ori.txt"))
fruit_ori = read_file(os.path.join("./과일/", "과일-ori.txt"))
godae_variants = [read_file(os.path.join("./고대/", f"고대-{i}.txt")) for i in range(1, 6)]
fruit_variants = [read_file(os.path.join("./과일/", f"과일-{i}.txt")) for i in range(1, 6)]

In [ ]:
train_examples = []

# ➤ 긍정쌍 (문장 의미 같음)
for i in range(len(godae_ori)):
    for variant in godae_variants:
        train_examples.append(InputExample(texts=[godae_ori[i], variant[i]], label=1.0))
    for variant in fruit_variants:
        train_examples.append(InputExample(texts=[fruit_ori[i], variant[i]], label=1.0))

# ➤ 부정쌍 (문맥/주제 다름: 고대 vs 과일)
for i in range(len(godae_ori)):
    for gv in godae_variants:
        for fv in fruit_variants:
            train_examples.append(InputExample(texts=[gv[i], fv[i]], label=0.0))

In [ ]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)
train_loss = losses.CosineSimilarityLoss(model=model)

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=50,
    warmup_steps=10,
    output_path="./output/ko-sbert-kykim-finetuned"
)

In [ ]:
sentences = [
    "서울의 모 회사에서 만든 반도체 설계도는 A 기술을 포함한다.",
    "수도권의 한 제조업체가 사용한 공정에는 A 기법이 응용됐다.."
]

embeddings = model.encode(sentences)
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
print(f"유사도: {similarity:.4f}")

In [ ]:
sentences = [
    "비록 영화에 대두한 것처럼 생물 병기 수준까지는 아니지만, 실제 고대 생물들 대부분 역시도 현대 생물보다 무섭던 맹수들인 경우가 대부분였기에 너무 근거없는 클리셰는 아닌 셈이다.",
    "실제 고대 생물들 상당수 역시도 현대 생물보다 무시무시했던 맹수들인 경우가 상당수였다. 아주 근거없는 클리셰는 아닌 셈이다."
]

embeddings = model.encode(sentences)
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
print(f"유사도: {similarity:.4f}")

In [ ]:
sentences = [
    "사실 각 지역 간에 정형화된 패턴의 고대는 존재하지 않는다. 중세 또한 존재하지 않는다.",
    "해당 뜻에 따르면 '풀에서 나는 식용 가능한 실과'는 엄밀히 말해 과실이 아니다."
]

embeddings = model.encode(sentences)
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
print(f"유사도: {similarity:.4f}")